In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

def processar_video_roi(video_path, output_path):

    print("Carregando modelo YOLOv8 Medium (mais preciso)...")
    model = YOLO("yolov8m.pt") 

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    roi_points = np.array([
        [618, 605], 
        [1137, 600], 
        [1417, 1047], 
        [481, 1042]
    ], np.int32)
    roi_points = roi_points.reshape((-1, 1, 2))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (width, height))

    print(f"Processando: {video_path}...")
    print(f"Resolução: {width}x{height}")

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model.predict(frame, conf=0.3, classes=[0], verbose=False)
        
        boxes = results[0].boxes.xyxy.cpu().numpy()
        pessoas_na_roi = 0

        cv2.polylines(frame, [roi_points], isClosed=True, color=(255, 0, 0), thickness=2)

        for box in boxes:
            x1, y1, x2, y2 = box
            cx = int((x1 + x2) / 2)
            cy = int(y2) 

            is_inside = cv2.pointPolygonTest(roi_points, (cx, cy), False)

            if is_inside >= 0:
                pessoas_na_roi += 1
                # Caixa Verde (Dentro)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            else:
                # Caixa Vermelha (Fora)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)

        # Contador visual
        cv2.rectangle(frame, (20, 20), (350, 80), (0, 0, 0), -1)
        cv2.putText(frame, f"Pessoas na Area: {pessoas_na_roi}", (35, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        out.write(frame)
        
        # Mostra uma versão menor na tela para não ocupar tudo se o vídeo for 4K
        frame_small = cv2.resize(frame, (1024, int(1024 * height / width)))
        cv2.imshow("Monitoramento (Preview Reduzido)", frame_small)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Concluído! Verifique o arquivo ")

if __name__ == "__main__":
    # ATENÇÃO: Mudei a extensão de saída para .avi
    processar_video_roi("runners.mp4", "resultado_roi.mp4")

In [ ]:
import cv2
from lightgbm import cv

video = cv.VideoCapture("runners.mp4")

body_cascade = cv2.CascadeClassifier("haarcascade_fullbody.xml")

min_size = (30, 60) #ajustar de acordo com resolução do vídeo]

while True:
    ret, frame = video.read()
    if not ret:
        break

    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # bodies = body_cascade.detectMultiScale(gray, 1.1, 3, minSize=min_size)

    # for (x, y, w, h) in bodies:
    #     cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # cv2.imshow("Video", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

frame_resized = cv2.resize(frame, dsize=(640, 480))


gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

gray_blurred = cv2.GaussianBlur(gray_frame, ksize=(5, 5), sigmaX=0)

bodies = body_cascade.detectMultiScale(gray_blurred, scaleFactor=1.1, minNeighbors=3, minSize=min_size)

for (x, y, w, h) in bodies:
    cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow(winname="People Detection", mat=frame_resized)

if cv2.waitKey(0) & 0xFF == ord('q'):
    break

video.release()
cv2.destroyAllWindows()



In [ ]:
import cv2

# 1. Use cv2 directly, removed the lightgbm import
video = cv2.VideoCapture("runners.mp4")

# Ensure the video opened successfully
if not video.isOpened():
    print("Error opening video file")
    exit()

body_cascade = cv2.CascadeClassifier("haarcascade_fullbody.xml")

# Adjusted min_size. Since we are resizing the frame to 640x480, 
# (30, 60) is a reasonable starting point.
min_size = (30, 60) 

while True:
    ret, frame = video.read()
    
    # Check if the frame was read correctly
    if not ret:
        break

    # 2. ALL processing logic must be INSIDE the while loop
    
    # Resize the frame to speed up detection and standardize input
    frame_resized = cv2.resize(frame, dsize=(640, 480))

    # Convert to Grayscale
    gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # Blur to reduce noise and false positives
    gray_blurred = cv2.GaussianBlur(gray_frame, ksize=(5, 5), sigmaX=0)

    # Detect bodies
    # ScaleFactor 1.1 means the image is reduced by 10% at each scale
    body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_fullbody.xml")
    bodies = body_cascade.detectMultiScale(
        gray_blurred, 
        scaleFactor=1.1, 
        minNeighbors=3, 
        minSize=min_size
    )

    # Draw rectangles
    for (x, y, w, h) in bodies:
        cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the frame
    cv2.imshow(winname="People Detection", mat=frame_resized)

    # 3. Use waitKey(1) for video playback (1ms delay), not waitKey(0)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()